# ExtTask GNN的模型训练

## 1 知识回顾

- GNN通用框架包括邻域信息转换、邻域信息集合、跨层连接、输入图和特征的增广、学习目标

- 单个GNN层：
    - 信息计算：$m_u^{(l)} = \text{MSG}^{(l)} (h_u^{(l - 1)})$
    - 信息聚合：$h_v^{(l)} = \text{AGG}^{(l)} (\{ m_u^{(l)}, u \in N(v) \})$

- 图特征增强
    - 解决输入图缺乏特征问题：特征增强
- 图结构增强
    - 解决图太稀疏问题：添加虚拟节点或边
    - 解决图太密集问题：消息传递时只采样部分邻居节点进行传递
    - 解决图太大问题：计算embedding时，对子图进行采样

## 2 GNN的预测

- 预测头（Prediction Head）主要包括：节点级任务、边级别任务、图级别任务

- 节点预测：直接使用节点嵌入进行预测
$$
\hat{y}_v = \text{Head}_{\text{node}}(h_v^{(L)}) = W^{(H)} h_v^{(L)}
$$
其中，$W^{(H)} \in R^{k \times d}$将映射节点嵌入从$h_v^{(L)} \in R^d$到$\hat{y}_v \in R^k$，从而计算损失

- 边预测：
    - 串联+线性：$\hat{y}_{uv} = \text{Linear}(\text{Concat} (h_u^{(L)}, h_v^{(L)}))$
    - 点积：$\hat{y}_{uv} = (h_u^{(L)})^T h_v^{(L)}$，适用于`1-way`预测

- 图预测：使用图中的所有节点嵌入进行预测
    - $\hat{y}_G = \text{Head}_{\text{graph}} (\{ h_v^{(L)} \in R^d, \forall v \in G\})$，其中，$\text{Head}_{\text{graph}}(\cdot)$类似于GNN层中的聚合函数$\text{AGG}(\cdot)$
    - 全局平均池化：$\hat{y}_G = \text{Mean} ({h_v^{(L)}} \in R^d, \forall v \in G)$
    - 全局最大池化：$\hat{y}_G = \text{Max} ({h_v^{(L)}} \in R^d, \forall v \in G)$
    - 全局求和池化：$\hat{y}_G = \text{Sum} ({h_v^{(L)}} \in R^d, \forall v \in G)$

- 全局池化层的问题：以上的全局池化层都只适用于小规模的图形，在大图上使用会造成信息丢失
    - 解决方案：使用分层池化
    - 社群分层池化：将每个社群当作一层，聚合其中的所有节点信息，使用社群检测（监督学习）分成多个簇，再汇总社群的信息，为每个社群生成一个超级节点，根据社群之间的联系，不断聚合，得到一个超级节点为止，最后输入到预测头中

## 3 GNN的训练

### 3.1 监督学习 vs. 无监督学习

- 图上的监督学习：标签来自外部来源，监督标签来自特定用例
- 图上的无监督学习：信号来自图本身，使用自我监督学习的方式

### 3.2 分类 vs. 回归

- 分类：节点的标签$y^{i}$具有离散值
- 回归：节点的标签$y^{i}$具有连续值

### 3.3 损失

- 分类损失：使用交叉熵损失函数，预测$K$类的第$i$个数据点的类别
$$
\text{CE}(y^{(i)}, \hat{y}^{(i)}) = - \sum_{j = 1}^K y_i^{(i)} \log(\hat{y}_j^{(i)})
$$

- 回归损失：使用均方误差损失函数
$$
\text{MSE}(y^{(i)}, \hat{y}^{(i)}) = \sum_{j = 1}^K (y_j^{(i)} - \hat{y}_j^{(i)})^2
$$

### 3.4 评估

- 回归评估：
    - RMSE：$\displaystyle \sqrt{\sum_{i=1}^N \frac{(y^{(i)} - \hat{y}^{(i)})^2}{N}}$
    - MAE：$\displaystyle \frac{\displaystyle \sum_{i=1}^N |y^{(i)} - \hat{y}^{(i)}|}{N}$

- 分类评估：
    - Accuracy：$\displaystyle \frac{1[\arg \max (\hat{y}^{(i)}) = y^{(i)}]}{N}$

- 二分类评估：
    - Accuracy：$\displaystyle \frac{\text{TP} + \text{TN}}{\text{TP} + \text{TN} + \text{FP} + \text{FN}}$
    - Percision：$\displaystyle \frac{\text{TP}}{\text{TP} + \text{FP}}$
    - Recall：$\displaystyle \frac{\text{TP}}{\text{TP} + \text{FN}}$
    - F1-Score：$\displaystyle \frac{2\text{P} \times \text{R}}{\text{P} + \text{R}}$

## 4 数据集划分

- 固定拆分：
    - 训练集：用于优化GNN参数
    - 验证集：用于调整超参数
    - 测试集：用于评估最终模型的性能
- 随机拆分：使用不同的随机种子，计算拆分训练集的平均模型性能

- 信息泄露问题：由于节点之间相互连接，会从其他节点收集信息，造成信息泄露

- 解决方案：
    1. 直推式学习（Transductive setting）：只拆分节点标签，保持图的结构不变
    2. 归纳式学习（Inductive setting）：删除拆分出的数据集之间连接的边

- 直推式学习：训练/验证/测试集在同一张图上
    - 数据集由单个图组成
    - 只拆分标签
    - 仅适用于节点/边预测任务
- 归纳式学习：训练/验证/测试集在不同的图上
    - 数据集由多个图组成
    - 每个拆分只能观察拆分内的图
    - 适用于节点/边/图任务

## 5 连接预测

- 预测步骤：
    1. 将原始图中的边，分为两类：消息边（用于GNN中的消息传递）、监督边（用于计算目标）
    2. 图中仅保留消息边，移除监督边
    3. 将边拆分为训练、验证、测试：归纳连接预测拆分、直推连接预测分割

## 6 本章总结

本次任务，主要介绍了GNN的模型训练，包括：

- 模型预测：包括节点级任务、边级别任务、图级别任务，使用社群分层池化方式解决图预测任务
- 模型训练：图上的监督和无监督学习，回归训练和分类训练、损失函数、模型评估
- 数据集划分：直推式学习和归纳式学习
- 连接预测：将边分为消息边和监督边，归纳连接预测拆分，直推连接预测分割